<a href="https://colab.research.google.com/github/smjacks4/510DW/blob/main/FINAL%2C_FINAL_of_Module_4_Searching_for_Codewords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports libraries

In [19]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import re
import os


Grabs and Gets

In [34]:
def grabarticle(thisurl):
    mod4 = requests.get(thisurl)
    bs = BeautifulSoup(mod4.text, 'html.parser')
    story_body = bs.find(class_='dialog-off-canvas-main-canvas')
    paragraphs = story_body.find_all('p') if story_body else []
    article = '\n\n'.join(x.text for x in paragraphs)

    title_tag = bs.find('title')
    author_tag = bs.find(itemprop='author')
    date_tag = bs.find('time')

    title = title_tag.text.replace(' | ASU News', '').strip() if title_tag else None
    author = author_tag.text.strip() if author_tag else None
    date = date_tag.get('datetime', date_tag.text).strip() if date_tag else None

    return {
        'URL': thisurl,
        'Author': author,
        'Title': title,
        'Date': date,
        'Article': article
    }


In [35]:
csv_path = 'ASUNewsArticlesFull.csv'

if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    print(f"Loaded {len(df)} existing articles from {csv_path}")
else:
    df = pd.DataFrame(columns=['URL', 'Date', 'Title', 'Author', 'Article'])
    print("No existing file found — starting fresh.")


Loaded 0 existing articles from EmptyColumns.csv


In [36]:
thispage = requests.get('https://news.asu.edu/')
bs = BeautifulSoup(thispage.text,'html.parser')
latest = bs.find(class_='dialog-off-canvas-main-canvas')
links = latest.find_all('a')


Identifying Codewords

In [37]:
codeword_hits = []  # <-- store only codeword matches here
codewords = ['medicine', 'breakthrough', 'upcoming', 'innovative']

Iterating, getting, and looking at the results

In [38]:
pattern = re.compile(r'https://news\.asu\.edu/\d{2}.*')

for eachlink in links:
    href = eachlink.get('href')
    if not href:
        continue

    if href.startswith('http'):
        full_url = href
    else:
        full_url = 'https://news.asu.edu' + href

    if not full_url.startswith('https://news.asu.edu/'):
        continue

    if not pattern.match(full_url):
        continue

    if not df['URL'].fillna('').astype(str).str.contains(full_url, regex=False).any():
        print('getting', full_url)
        story = grabarticle(full_url)
        sleep(15)

        story_df = pd.DataFrame([story])
        df = pd.concat([df, story_df], ignore_index=True)

        article_text = story.get('Article', '').lower()
        found_words = [word for word in codewords if word in article_text]

        if found_words:
            codeword_hits.append({
                'URL': full_url,
                'Found_Codewords': ', '.join(found_words)
            })

getting https://news.asu.edu/20251022-science-and-technology-campus-company-how-research-sparked-new-water-tech-startup
getting https://news.asu.edu/20251106-health-and-medicine-life-service-how-asus-cheryl-schmidt-turned-duty-legacy
getting https://news.asu.edu/20251103-science-and-technology-smarter-fuel-pipelines-safer-communities
getting https://news.asu.edu/20251030-university-news-asu-ranked-among-world-best-graduate-employability
getting https://news.asu.edu/2025022-science-and-technology-asu-computer-scientists-ai-jobs-and-road-ahead
getting https://news.asu.edu/20250915-science-and-technology-asu-student-works-protect-african-penguins-local-aquarium
getting https://news.asu.edu/20251110-sun-devil-community-serving-those-who-are-built-serve
getting https://news.asu.edu/20251107-university-news-asu-state-arizona-launch-initiative-power-public-workforce
getting https://news.asu.edu/20251106-law-journalism-and-politics-first-amendment-academy-empowers-citizens-knowledge-five-freed

In [39]:
display(df)

,URL,Date,Title,Author,Article
0,https://news.asu.edu/20251022-science-and-tech...,2025-10-22T15:53:26-07:00,From campus to company: How research sparked a...,By Faith Kearns,Enrique Vivoni's startup Tributary helps group...
1,https://news.asu.edu/20251106-health-and-medic...,2025-11-06T17:24:37-07:00,A life of service: How ASU’s Cheryl Schmidt tu...,By Marshall Terrill,The Arizona Veterans Hall of Fame inductee has...
2,https://news.asu.edu/20251103-science-and-tech...,2025-11-03T14:31:39-07:00,"Smarter fuel pipelines, safer communities",By Kelly deVos,ASU researchers are using robotics and AI to r...
3,https://news.asu.edu/20251030-university-news-...,2025-10-30T09:00:24-07:00,ASU named among the world’s best universities ...,By Marshall Terrill,"Holding strong, the university retains its No...."
4,https://news.asu.edu/2025022-science-and-techn...,2025-09-23T02:02:53-07:00,The moment to lead: ASU computer scientists on...,By Kelly deVos,"In this Q&A, leaders share why today’s challen..."
5,https://news.asu.edu/20250915-science-and-tech...,2025-09-15T13:58:34-07:00,ASU student works to protect African penguins ...,By Laura Fields,Neuroscience major Makaela Ross applies her st...
6,https://news.asu.edu/20251110-sun-devil-commun...,2025-11-10T15:59:22-07:00,Serving those who are built to serve,By Meenah Rincon,ASU Online helps military-affiliated students ...
7,https://news.asu.edu/20251107-university-news-...,2025-11-07T23:09:01-07:00,"ASU, state of Arizona launch initiative to pow...",By Samantha Becker,State employees can now access dozens of ASU c...
8,https://news.asu.edu/20251106-law-journalism-a...,2025-11-06T16:18:40-07:00,New First Amendment Academy empowers citizens ...,By Marshall Terrill,"Free courses, flash quizzes available to anyon..."
9,https://news.asu.edu/20251110-health-and-medic...,2025-11-10T16:16:37-07:00,Untangling the influence of cannabis use on bi...,By Kimberlee D’Ardenne,Study finds cannabis use decreases likelihood ...


Cleaning and Saving ASU News Full Data

In [40]:
df = df[df['Date'].astype(str).str.strip() != '']
df.to_csv('ASUNewsArticlesFull.csv', index=False)
print(f"\n Saved {len(df)} total articles to {csv_path}")


 Saved 37 total articles to EmptyColumns.csv


Saving codeword matches to their own file

In [41]:
if codeword_hits:
    df_hits = pd.DataFrame(codeword_hits)
    df_hits.to_csv('ASU_codewords_found.csv', index=False)
    print(f" Saved {len(df_hits)} codeword matches to ASU_codewords_found.csv")
else:
    print("No codewords found in any article.")

 Saved 15 codeword matches to ASU_codewords_found.csv


END